In [10]:
import numpy as np
import pandas as pd
import sys
import os

In [2]:
pathToExcel = "E:\Github\Ladder-Iterative-Loadflow\Documentations\Datasets\IEEE 33 CDF (Updated)_2.xlsx"
tolerance = 10e-4
#arrayToUse = dataParser(pathToExcel)
#print(arrayToUse)

In [9]:
#def dataParser(path):
path = pathToExcel
readInput = pd.read_excel(path)
numpyConversion = readInput.to_numpy()
checker = checkIfAllTablesExist(readInput)

if checker == False:
    raise ValueError("Excel does not contain Bus or Branch data, please double check and try again")

busData = extractData(numpyConversion, "BUS DATA FOLLOWS")
branchData = extractData(numpyConversion, "BRANCH DATA FOLLOWS")




# To display
x = pd.DataFrame(branchData)
display(x)




,0,1,2,3,4
0,1.0,2.0,0.0575,0.0293,0.0
1,2.0,3.0,0.3076,0.1567,0.0
2,3.0,4.0,0.2284,0.1163,0.0
3,4.0,5.0,0.2378,0.1211,0.0
4,5.0,6.0,0.5110,0.4411,0.0
5,6.0,7.0,0.1168,0.3861,0.0
6,7.0,8.0,0.4439,0.1467,0.0
7,8.0,9.0,0.6426,0.4617,0.0
8,9.0,10.0,0.6514,0.4617,0.0
9,10.0,11.0,0.1227,0.0406,0.0


In [3]:
def checkIfAllTablesExist(arr):
    if "BUS DATA FOLLOWS" or "BRANCH DATA FOLLOWS" in arr:
        return True
    else:
        return False

In [4]:
#a = np.where(numpyConversion=="BRANCH DATA FOLLOWS")
#print(a[0])
#print(a.shape)
def extractData(npArr, txt):

    lenOfArray = npArr.shape[0]
    widthOfArray = npArr.shape[1]
    forLoopStartValue = int(np.where(npArr==txt)[0])+1
    for i in range(forLoopStartValue, lenOfArray):
        if npArr[i][0] == -999:
            newArray = npArr[forLoopStartValue:i]
            break

    if txt == "BUS DATA FOLLOWS":
        arrayToReturn = np.zeros((newArray.shape[0],4))
        arrayToReturn[:,0] = newArray[:,0]
        arrayToReturn[:,1] = newArray[:,7]
        arrayToReturn[:,2] = newArray[:,8]
        arrayToReturn[:,3] = newArray[:,11]
    elif txt == "BRANCH DATA FOLLOWS":
        arrayToReturn = np.zeros((newArray.shape[0],5))
        arrayToReturn[:,0] = newArray[:,0]
        arrayToReturn[:,1] = newArray[:,1]
        arrayToReturn[:,2] = newArray[:,6]
        arrayToReturn[:,3] = newArray[:,7]
        arrayToReturn[:,4] = 0
        
    
    return arrayToReturn


In [15]:
values, counts = np.unique(branchData[:,0], return_counts=True)
print(values)
print(counts)

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 19.
 20. 21. 23. 24. 26. 27. 28. 29. 30. 31. 32.]
[1 2 2 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [8]:
newData = branchData
lenOfArray = newData.shape[0]
newBranchDataArr = np.zeros((newData.shape[0],4))
n=0
duplicateRan = False
for i in range(0, lenOfArray):
    dup = np.where(newData[:,0]==newData[i,0])
    if len(dup[0])==1 and (newData[i,4]==0):
        newBranchDataArr[n,0] = newData[i,0]
        newBranchDataArr[n,1] = newData[i,1]
        newBranchDataArr[n,2] = newData[i,2]
        newBranchDataArr[n,3] = newData[i,3]
        newData[i,4] = 1
        n=n+1            
    else:
        for j in dup[0]:
            k = j
            if (j > i) and (newData[k,4]==0):
                
                while(1):
                    if k+1 == lenOfArray:
                        if (newData[k,0]==newData[k-1,1]):
                            newBranchDataArr[n,0] = newData[k,0]
                            newBranchDataArr[n,1] = newData[k,1]
                            newBranchDataArr[n,2] = newData[k,2]
                            newBranchDataArr[n,3] = newData[k,3]
                            newData[k,4] = 1
                            n = n+1
                            k = k+1
                        break
                    if (newData[k,1]==newData[k+1,0]):
                        newBranchDataArr[n,0] = newData[k,0]
                        newBranchDataArr[n,1] = newData[k,1]
                        newBranchDataArr[n,2] = newData[k,2]
                        newBranchDataArr[n,3] = newData[k,3]
                        newData[k,4] = 1
                        n = n+1
                        k = k+1
                    elif (newData[k,1]!=newData[k+1,0]) and (newBranchDataArr[n-1,0] != newData[k,0]):
                        newBranchDataArr[n,0] = newData[k,0]
                        newBranchDataArr[n,1] = newData[k,1]
                        newBranchDataArr[n,2] = newData[k,2]
                        newBranchDataArr[n,3] = newData[k,3]
                        newData[k,4] = 1
                        n = n+1
                        k = k+1
                        break
                duplicateRan = True
    if duplicateRan == True and (newData[i,4]==0):
        newBranchDataArr[n,0] = newData[i,0]
        newBranchDataArr[n,1] = newData[i,1]
        newBranchDataArr[n,2] = newData[i,2]
        newBranchDataArr[n,3] = newData[i,3]
        newData[i,4] = 1
        n=n+1
        duplicateRan = False
        
# To display
x = pd.DataFrame(newBranchDataArr)
display(x)

,0,1,2,3
0,1.0,2.0,0.0575,0.0293
1,2.0,19.0,0.1023,0.0976
2,19.0,20.0,0.9385,0.8457
3,20.0,21.0,0.2555,0.2985
4,21.0,22.0,0.4423,0.5848
5,2.0,3.0,0.3076,0.1567
6,3.0,23.0,0.2815,0.1924
7,23.0,24.0,0.5603,0.4424
8,24.0,25.0,0.5590,0.4374
9,3.0,4.0,0.2284,0.1163
